## Neo4j Database Creation

In [1]:
!pip install py2neo

In [2]:
import py2neo
import numpy as np

In [3]:
from py2neo import Graph
from py2neo import Node
from py2neo import Relationship

In [4]:
graph = Graph("bolt://localhost:7687", auth=("neo4j", "launch"))

In [5]:
import pandas as pd

In [6]:
courses = pd.read_csv('courses.csv', encoding = "ISO-8859-1", engine='python') 
fam_members = pd.read_csv('familyMembers.csv', encoding = "ISO-8859-1", engine='python') 
group_members = pd.read_csv('groupMembers.csv', encoding = "ISO-8859-1", engine='python') 
pets = pd.read_csv('pets.csv', encoding = "ISO-8859-1", engine='python') 

In [7]:
group_members.columns

Index(['id', 'name', 'birthday', 'birthCity', 'hometown', 'age',
       'siblingCount', 'sibling1', 'sibling2', 'sibling3', 'petCount', 'pet1',
       'pet2', 'course1', 'course2', 'course3', 'course4', 'course5',
       'course6', 'course7', 'major', 'favIceCream', 'inState'],
      dtype='object')

In [8]:
pets['since'] = [2016,2020,2008,2008,2019]
pets.head()

,id,name,species,breed,gender,age,ownerGroup,color,since
0,1,King Louie,Dog,Unknown,M,5.00,1,white,2016
1,2,Taro,Dog,Border Collie mix,M,0.58,3,Brown,2020
2,3,Blacky,Cat,NaN,M,13.00,4,Black,2008
3,4,Jacky,Cat,NaN,M,13.00,4,Brown,2008
4,5,Leo,Dog,German Shepherd,M,2.80,2,Black,2019


In [9]:
# Group Member Node Creation
for i in range(len(group_members)):
    row = group_members.iloc[i] 
    person = Node("Person", name = row['name'], birthday=row['birthday'], birthCity = row['birthCity'], hometown = row['hometown'], age = int(row['age']), siblingCount = int(row['siblingCount']), major = row['major'], inState = int(row['inState']))
    graph.create(person)

# Pet Node Creation
for i in range(len(pets)):
    row = pets.iloc[i] 
    pet = Node("Pet", name = row['name'], species=row['species'], breed = row['breed'], gender = row['gender'], age = int(row['age']), color = row['color'])
    graph.create(pet)
    
# Fam Members Node Creation
for i in range(len(fam_members)):
    row = fam_members.iloc[i] 
    familyMember = Node("familyMember", name = row['name'], age= int(row['age']), gender = row['gender'], birthday = row['birthday'], petCount = int(row['petCount']), enrolledAtUVA = bool(row['enrolledAtUVA']), favIceCream=str(row['favIceCream']))
    graph.create(familyMember)

# Courses Node Creation
for i in range(len(courses)):
    row = courses.iloc[i] 
    course = Node("Course", courseMnemonic = row['course mnemonic'], courseNumber= int(row['course number']), department = row['department'], classTitle = row['class title'], professor = row['professor'], meetingDays = (row['meeting day(s)']), classType = row['type'], classCapacity = int(row['class capacity']), courseDescription = row['course description'])
    graph.create(course)


                  

In [10]:
from py2neo import Graph, NodeMatcher
matcher = NodeMatcher(graph)

In [11]:
fam_members.columns

Index(['id ', 'relation', 'name', 'age', 'gender', 'birthday', 'favIceCream',
       'petCount', 'pet1Name', 'pet2Name', 'enrolledAtUVA', 'courses',
       'groupMemberRelative'],
      dtype='object')

In [12]:
Owns = Relationship.type("OWNS")
for i in range(len(group_members)):
    if not pd.isna(group_members['pet1'][i]):
        matchone = matcher.match("Person").where("_.name = '" + str(group_members.name[i]) + "'").first()
        thisPet = pets.loc[pets['id '] == group_members.pet1[i]].reset_index()
        matchtwo = matcher.match("Pet").where("_.name = '" + str(thisPet.name[0]) + "'").first()
        sinceDate = pets.loc[pets['id '] == group_members.pet1[i]].reset_index()
        relProp = {'since':int(sinceDate.since[0])}
        print(matchone, matchtwo)
        graph.create(Owns(matchone, matchtwo, **relProp))

(_54:Person {age: 19, birthCity: 'Boston, MA', birthday: '2001/06/11', hometown: 'Charlottesville, VA', inState: 1, major: 'Computer Science, Public Policy and Leadership', name: 'Anna Grace Calhoun', siblingCount: 3}) (_59:Pet {age: 5, breed: 'Unknown', color: 'white ', gender: 'M', name: 'King Louie', species: 'Dog'})
(_55:Person {age: 19, birthCity: 'Gainesville, FL', birthday: '2001/12/28', hometown: 'Fairfax, VA', inState: 1, major: 'Aerospace Engineering', name: 'Daniel Tohti', siblingCount: 2}) (_63:Pet {age: 2, breed: 'German Shepherd', color: 'Black', gender: 'M', name: 'Leo', species: 'Dog'})
(_56:Person {age: 24, birthCity: 'Virginia Beach, VA', birthday: '1996/09/03', hometown: 'Virginia Beach, VA', inState: 1, major: 'Psychology', name: 'Dare Nguyen', siblingCount: 3}) (_60:Pet {age: 0, breed: 'Border Collie mix', color: 'Brown', gender: 'M', name: 'Taro', species: 'Dog'})
(_57:Person {age: 7, birthCity: 'Reston, VA', birthday: '2014/05/30', hometown: 'Fairfax, VA', inStat

In [13]:
for i in range(len(group_members)):
    if not pd.isna(group_members['pet2'][i]):
        matchone = matcher.match("Person").where("_.name = '" + str(group_members.name[i]) + "'").first()
        thisPet = pets.loc[pets['id '] == group_members.pet2[i]].reset_index()
        matchtwo = matcher.match("Pet").where("_.name = '" + str(thisPet.name[0]) + "'").first()
        sinceDate = pets.loc[pets['id '] == group_members.pet2[i]].reset_index()
        relProp = {'since':int(sinceDate.since[0])}
        print(matchone, matchtwo)
        graph.create(Owns(matchone, matchtwo, **relProp))

(_57:Person {age: 7, birthCity: 'Reston, VA', birthday: '2014/05/30', hometown: 'Fairfax, VA', inState: 1, major: 'Mathematics, Computer Science', name: 'David Siamon', siblingCount: 1}) (_62:Pet {age: 13, breed: nan, color: 'Brown', gender: 'M', name: 'Jacky', species: 'Cat'})


In [14]:
fam_members.columns

Index(['id ', 'relation', 'name', 'age', 'gender', 'birthday', 'favIceCream',
       'petCount', 'pet1Name', 'pet2Name', 'enrolledAtUVA', 'courses',
       'groupMemberRelative'],
      dtype='object')

In [15]:
#Figure out the relationship between relative
def relationship_check(relation_input):
    if((relation_input.strip().lower() == 'sister') | (relation_input.strip().lower() == 'brother')):
        return 'sibling'
    if((relation_input.strip().lower()=='father') | (relation_input.strip().lower()=='mother')):
        return 'parent'
    if((relation_input.strip().lower()=='son') | (relation_input.strip().lower()=='daughter')):
        return 'child'

In [16]:
#Flips relationship for bi-directional
def relationship_flip(relation_input):
    if(relation_input=='sibling'):
        return 'sibling'
    if(relation_input=='parent'):
        return 'child'
    if(relation_input=='child'):
        return 'parent'

In [17]:
IsRelatedTo = Relationship.type("IS_RELATED_TO")
for i in range(len(fam_members)):
    matchone = matcher.match("familyMember").where("_.birthday= '" + str(fam_members.birthday[i]) + "'").first()
    relative = group_members.loc[group_members['id'] == fam_members.groupMemberRelative[i]].reset_index()
    matchtwo = matcher.match("Person").where("_.name = '" + str(relative.name[0]) + "'").first()
    relativeType = relationship_check(fam_members.relation[i])
    relProp = {'relation':str(relativeType)}
    graph.create(IsRelatedTo(matchone, matchtwo,**relProp))
    relProp2 = {'relation':str(relationship_flip(relativeType))}
    graph.create(IsRelatedTo(matchtwo,matchone,**relProp2))

In [18]:
courses.columns

Index(['id', 'course mnemonic', 'course number', 'department', 'class title',
       'professor', 'meeting day(s)', 'type', 'class capacity',
       'course description'],
      dtype='object')

In [19]:
courses.head()

,id,course mnemonic,course number,department,class title,professor,meeting day(s),type,class capacity,course description
0,0,APMA,3140,Applied Mathematics,Partial Differential Equations,Diana Morris,"M,W,F",Lecture,300,Partial differential equations that govern phy...
1,1,CS,3250,Computer Science,Software Testing,Upsorn Praphamontripong,"Tu, Th",Lecture,60,An introduction to testing for assuring softwa...
2,2,APMA,3501,Applied Mathematics,Spec Topics in APMA,Gianluca Guadagni,M,Lecture,75,Applies mathematical techniques to special pro...
3,3,LPPP,3200,Leadership and Public Policy,Introduction to Public Policy,NaN,M,Discussion,20,This course will introduce students to both th...
4,4,MAE,2040,Mechanical & Aerospace Engineering,Computer Aided Design,Shawn Russell,M,Lab,25,Communication through engineering graphics; en...


In [20]:
IsEnrolledIn = Relationship.type("IS_ENROLLED_IN")
#course1
for i in range(len(group_members)):
    course = group_members.loc[i,'course1']
    if(not pd.isna(course)):
        courseOb = courses.loc[course]
        courseOb.head()
        labels = {'classTitle':str(courseOb['class title']),'professor': str(courseOb['professor'])}
        matchone = matcher.match("Course").where(**labels).first()
        # matching courses to group members
        relProp = {'professor':str(matchone['professor']), 'meetingDays':str(matchone['meetingDays'])}
        matchtwo = matcher.match("Person").where("_.name = '" + str(group_members.loc[i,'name']) + "'").first()
        graph.create(IsEnrolledIn(matchtwo, matchone, **relProp))
#course2
for i in range(len(group_members)):
    course = group_members.loc[i,'course2']
    if(not pd.isna(course)):
        courseOb = courses.loc[course]
        labels = {'classTitle':str(courseOb['class title']), 'classCapacity': int(courseOb['class capacity'])}
        matchone = matcher.match("Course").where(**labels).first()
        # matching courses to group members
        relProp = {'professor':str(matchone['professor']), 'meetingDays':str(matchone['meetingDays'])}
        matchtwo = matcher.match("Person").where("_.name = '" + str(group_members.loc[i,'name']) + "'").first()
        graph.create(IsEnrolledIn(matchtwo, matchone, **relProp))
#course3
for i in range(len(group_members)):
    course = group_members.loc[i,'course3']
    if(not pd.isna(course)):
        courseOb = courses.loc[course]
        labels = {'classTitle':str(courseOb['class title']), 'professor': str(courseOb['professor'])}
        matchone = matcher.match("Course").where(**labels).first()        # matching courses to group members
        relProp = {'professor':str(matchone['professor']), 'meetingDays':str(matchone['meetingDays'])}
        matchtwo = matcher.match("Person").where("_.name = '" + str(group_members.loc[i,'name']) + "'").first()
        graph.create(IsEnrolledIn(matchtwo, matchone, **relProp))
#course4
for i in range(len(group_members)):
    course = group_members.loc[i,'course4']
    if(not pd.isna(course)):
        courseOb = courses.loc[course]
        labels = {'classTitle':str(courseOb['class title']), 'professor': str(courseOb['professor'])}
        matchone = matcher.match("Course").where(**labels).first()        # matching courses to group members
        relProp = {'professor':str(matchone['professor']), 'meetingDays':str(matchone['meetingDays'])}
        matchtwo = matcher.match("Person").where("_.name = '" + str(group_members.loc[i,'name']) + "'").first()
        graph.create(IsEnrolledIn(matchtwo, matchone, **relProp))
#course5
for i in range(len(group_members)):
    course = group_members.loc[i,'course5']
    if(not pd.isna(course)):
        courseOb = courses.loc[course]
        labels = {'classTitle':str(courseOb['class title']), 'professor': str(courseOb['professor'])}
        matchone = matcher.match("Course").where(**labels).first()
        # matching courses to group members
        relProp = {'professor':str(matchone['professor']), 'meetingDays':str(matchone['meetingDays'])}
        matchtwo = matcher.match("Person").where("_.name = '" + str(group_members.loc[i,'name']) + "'").first()
        graph.create(IsEnrolledIn(matchtwo, matchone, **relProp))
#course6
for i in range(len(group_members)):
    course = group_members.loc[i,'course6']
    if(not pd.isna(course)):
        courseOb = courses.loc[course]
        labels = {'classTitle':str(courseOb['class title']), 'professor': str(courseOb['professor'])}
        matchone = matcher.match("Course").where(**labels).first()
        # matching courses to group members
        relProp = {'professor':str(matchone['professor']), 'meetingDays':str(matchone['meetingDays'])}
        matchtwo = matcher.match("Person").where("_.name = '" + str(group_members.loc[i,'name']) + "'").first()
        graph.create(IsEnrolledIn(matchtwo, matchone, **relProp))
#course7
for i in range(len(group_members)):
    course = group_members.loc[i,'course7']
    if(not pd.isna(course)):
        courseOb = courses.loc[course]
        labels = {'classTitle':str(courseOb['class title']), 'professor': str(courseOb['professor'])}
        matchone = matcher.match("Course").where(**labels).first()
        # matching courses to group members
        relProp = {'professor':str(matchone['professor']), 'meetingDays':str(matchone['meetingDays'])}
        matchtwo = matcher.match("Person").where("_.name = '" + str(group_members.loc[i,'name']) + "'").first()
        graph.create(IsEnrolledIn(matchtwo, matchone, **relProp))

In [21]:
#Removing redundant professor/meetingDays properties
graph.run("MATCH (n:Course) REMOVE n.professor, n.meetingDays")

(No data)

In [22]:
#FamMembers connecting to respective Pets
for i in range(len(fam_members)):
    if not pd.isna(fam_members['pet1Name'][i]):
        matchone = matcher.match("familyMember").where("_.name = '" + str(fam_members.name[i]) + "'").first()
        thisPet = pets.loc[pets['id '] == fam_members.pet1Name[i]].reset_index()
        matchtwo = matcher.match("Pet").where("_.name = '" + str(thisPet.name[0]) + "'").first()
        sinceDate = pets.loc[pets['id '] == fam_members.pet1Name[i]].reset_index()
        relProp = {'since':int(sinceDate.since[0])}
        graph.create(Owns(matchone, matchtwo, **relProp))

In [23]:
for i in range(len(fam_members)):
    if not pd.isna(fam_members['pet2Name'][i]):
        matchone = matcher.match("familyMember").where("_.name = '" + str(fam_members.name[i]) + "'").first()
        thisPet = pets.loc[pets['id '] == fam_members.pet2Name[i]].reset_index()
        matchtwo = matcher.match("Pet").where("_.name = '" + str(thisPet.name[0]) + "'").first()
        sinceDate = pets.loc[pets['id '] == fam_members.pet2Name[i]].reset_index()
        relProp = {'since':int(sinceDate.since[0])}
        graph.create(Owns(matchone, matchtwo, **relProp))

## Queries

In [26]:
#6 Deg of Danny Devito
graph.run("MATCH (n:Person {name:'Danny Devito'})-[*1..3]-(m) RETURN n, m")

#Pet to Danny Devito
graph.run("MATCH sPath = shortestPath((n:Person {name:'Danny Devito'})-[*]-(m:Pet {name:'King Louie'})) RETURN sPath")

#Parent of CS on Monday
graph.run("MATCH (n:familyMember)-[:IS_RELATED_TO {relation:'parent'}]-(m:Person)-[e:IS_ENROLLED_IN]-(c:Course {classTitle:'Program & Data Representation'}) WHERE e.meetingDays CONTAINS 'M' RETURN n,m,c")

#Pet with Family Members that have the same Favorite Ice Cream
graph.run("MATCH (n:familyMember)-[:OWNS]-(p:Pet)-[:OWNS]-(m:familyMember) WHERE n.favIceCream=m.favIceCream RETURN n,p,m")

(No data)

In [25]:
#graph.run("match (n) detach delete n")